In [4]:
import os
import duckdb
import pandas as pd
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas as pd
import numpy as np

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV

### ER - BEGIN ###
import pickle
### ER - END ###

In [5]:
# file_path1 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/X_test_enc.parquet"
# file_path2 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/X_train_enc.parquet"
# file_path3 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/y_test.parquet"
# file_path4 = r"/home/kmohseni/private/DSC288R_Capstone/data/final/y_train.parquet"

### ER - BEGIN ###
file_path1 = "../data/final/test_data.parquet"
file_path2 = "../data/final/train_data.parquet"
### ER - END ###

test_data = pd.read_parquet(file_path1)
train_data = pd.read_parquet(file_path2)

In [6]:
test_data

,FlightDate,DayOfWeek,Month,Airline,Origin,Dest,CRSDepTime,CRSArrTime,AirTime,Distance,...,weekly_avg_dest_snow,monthly_avg_origin_snow,monthly_avg_dest_snow,weekly_avg_origin_snwd,weekly_avg_dest_snwd,monthly_avg_origin_snwd,monthly_avg_dest_snwd,weekly_avg_origin_depdelayminutes,monthly_avg_origin_depdelayminutes,DepDel15
0,2019-04-01,1,4,0.571607,0.581715,0.525375,2105,5,94.0,775.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,32.714286,29.100000,1.0
1,2019-02-27,3,2,0.458958,0.499612,0.536869,1210,1409,93.0,643.0,...,0.000000,0.0,9.666667,0.0,0.000000,0.0,52.666667,1.857143,1.133333,0.0
2,2022-03-10,4,3,0.501947,0.499612,0.496033,725,857,125.0,862.0,...,32.571429,0.0,29.733333,0.0,7.142857,0.0,40.000000,1.571429,33.833333,0.0
3,2020-02-29,6,2,0.456279,0.550683,0.541970,1640,1750,45.0,237.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.285714,28.600000,0.0
4,2019-07-25,4,7,0.456279,0.481732,0.496487,741,810,52.0,264.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,8.633333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1526291,2021-07-19,1,7,0.618388,0.526832,0.529108,805,1058,149.0,1041.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,21.571429,40.466667,0.0
1526292,2019-11-27,3,11,0.515444,0.550683,0.544548,1020,1435,165.0,1389.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,48.428571,31.966667,1.0
1526293,2019-08-27,2,8,0.618388,0.482008,0.529027,931,1123,81.0,636.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,93.000000,34.700000,1.0
1526294,2021-11-12,5,11,0.596937,0.466690,0.525375,1756,2032,83.0,619.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,24.571429,26.300000,0.0


In [7]:
print(test_data.columns)

Index(['FlightDate', 'DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest',
       'CRSDepTime', 'CRSArrTime', 'AirTime', 'Distance', 'DepDelayMinutes',
       'AirTimeCategory', 'TimeofDay', 'DayOfYear', 'Month_sin', 'Month_cos',
       'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear_sin', 'DayOfYear_cos',
       'CRSDepTime_sin', 'CRSDepTime_cos', 'Holiday_Indicator', 'Near_Holiday',
       'Origin_PRCP', 'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX',
       'Origin_TMIN', 'Dest_PRCP', 'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX',
       'Dest_TMIN', 'weekly_avg_origin_tmin', 'weekly_avg_dest_tmin',
       'monthly_avg_origin_tmin', 'monthly_avg_dest_tmin',
       'weekly_avg_origin_tmax', 'weekly_avg_dest_tmax',
       'monthly_avg_origin_tmax', 'monthly_avg_dest_tmax',
       'weekly_avg_origin_prcp', 'weekly_avg_dest_prcp',
       'monthly_avg_origin_prcp', 'monthly_avg_dest_prcp',
       'weekly_avg_origin_snow', 'weekly_avg_dest_snow',
       'monthly_avg_origin_snow', 'monthly_avg_dest_snow',


In [8]:
print(train_data.columns)

Index(['FlightDate', 'DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest',
       'CRSDepTime', 'CRSArrTime', 'AirTime', 'Distance', 'DepDelayMinutes',
       'AirTimeCategory', 'TimeofDay', 'DayOfYear', 'Month_sin', 'Month_cos',
       'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear_sin', 'DayOfYear_cos',
       'CRSDepTime_sin', 'CRSDepTime_cos', 'Holiday_Indicator', 'Near_Holiday',
       'Origin_PRCP', 'Origin_SNOW', 'Origin_SNWD', 'Origin_TMAX',
       'Origin_TMIN', 'Dest_PRCP', 'Dest_SNOW', 'Dest_SNWD', 'Dest_TMAX',
       'Dest_TMIN', 'weekly_avg_origin_tmin', 'weekly_avg_dest_tmin',
       'monthly_avg_origin_tmin', 'monthly_avg_dest_tmin',
       'weekly_avg_origin_tmax', 'weekly_avg_dest_tmax',
       'monthly_avg_origin_tmax', 'monthly_avg_dest_tmax',
       'weekly_avg_origin_prcp', 'weekly_avg_dest_prcp',
       'monthly_avg_origin_prcp', 'monthly_avg_dest_prcp',
       'weekly_avg_origin_snow', 'weekly_avg_dest_snow',
       'monthly_avg_origin_snow', 'monthly_avg_dest_snow',


In [9]:
# Drop unnecessary or repetitive columns
X_train_class = train_data.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'DepDel15', 'DepDelayMinutes'])
X_test_class = test_data.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'DepDel15', 'DepDelayMinutes'])
y_train_class = train_data['DepDel15']
y_test_class = test_data['DepDel15']

X_train_reg = train_data.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'DepDelayMinutes', 'DepDel15'])
X_test_reg = test_data.drop(columns=['TimeofDay', 'AirTimeCategory', 'FlightDate', 'DepDelayMinutes', 'DepDel15'])
y_train_reg = train_data['DepDelayMinutes']
y_test_reg = test_data['DepDelayMinutes']

In [10]:
print(X_train_class.columns)
print(X_train_reg.columns)
print(X_test_class.columns)
print(X_test_reg.columns)

Index(['DayOfWeek', 'Month', 'Airline', 'Origin', 'Dest', 'CRSDepTime',
       'CRSArrTime', 'AirTime', 'Distance', 'DayOfYear', 'Month_sin',
       'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'DayOfYear_sin',
       'DayOfYear_cos', 'CRSDepTime_sin', 'CRSDepTime_cos',
       'Holiday_Indicator', 'Near_Holiday', 'Origin_PRCP', 'Origin_SNOW',
       'Origin_SNWD', 'Origin_TMAX', 'Origin_TMIN', 'Dest_PRCP', 'Dest_SNOW',
       'Dest_SNWD', 'Dest_TMAX', 'Dest_TMIN', 'weekly_avg_origin_tmin',
       'weekly_avg_dest_tmin', 'monthly_avg_origin_tmin',
       'monthly_avg_dest_tmin', 'weekly_avg_origin_tmax',
       'weekly_avg_dest_tmax', 'monthly_avg_origin_tmax',
       'monthly_avg_dest_tmax', 'weekly_avg_origin_prcp',
       'weekly_avg_dest_prcp', 'monthly_avg_origin_prcp',
       'monthly_avg_dest_prcp', 'weekly_avg_origin_snow',
       'weekly_avg_dest_snow', 'monthly_avg_origin_snow',
       'monthly_avg_dest_snow', 'weekly_avg_origin_snwd',
       'weekly_avg_dest_snwd', 'monthly

# Classification

## Histogram-Based Gradient Boosting Classification Tree

In [11]:
# Initialize the HistGradientBoosting classifier
hgb_classifier = HistGradientBoostingClassifier(random_state=42)

# Train the model
hgb_classifier.fit(X_train_class, y_train_class)

HistGradientBoostingClassifier(random_state=42)

In [12]:
### ER - BEGIN ###
with open("../models/hgb_class.pkl", "wb") as file:
    pickle.dump(hgb_classifier, file)

In [13]:
with open("../models/hgb_class.pkl", "rb") as file:
    hgb_classifier = pickle.load(file)
### ER - END ###

In [14]:
# Make predictions
y_pred_hgb = hgb_classifier.predict(X_test_class)

In [15]:
# Print classification report
print("HistGradientBoosting Classification Report:")
print(classification_report(y_test_class, y_pred_hgb))

HistGradientBoosting Classification Report:
              precision    recall  f1-score   support

         0.0       0.83      0.74      0.78    763302
         1.0       0.77      0.85      0.81    762994

    accuracy                           0.80   1526296
   macro avg       0.80      0.80      0.80   1526296
weighted avg       0.80      0.80      0.80   1526296



# Regression

## Linear Regression

In [16]:
# Impute missing values with the median (or use 'mean' or 'most_frequent')
imputer = SimpleImputer(strategy="median")

# Fit and transform the training set
X_train_reg = imputer.fit_transform(X_train_reg)
X_test_reg = imputer.transform(X_test_reg)  # Use same imputer for test data

In [17]:
X_train_reg = X_train_reg.astype(np.float32)
X_test_reg = X_test_reg.astype(np.float32)
y_train_reg = y_train_reg.astype(np.float32)
y_test_reg = y_test_reg.astype(np.float32)

In [18]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_reg, y_train_reg)

LinearRegression()

In [19]:
### ER - BEGIN ###
with open("../models/lin_reg.pkl", "wb") as file:
    pickle.dump(lin_reg, file)

In [20]:
with open("../models/lin_reg.pkl", "rb") as file:
    lin_reg = pickle.load(file)
### ER - END ###

In [21]:
y_pred_linreg = lin_reg.predict(X_test_reg)

print("Linear Regression MAE:", mean_absolute_error(y_test_reg, y_pred_linreg))
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_linreg)))
print("Linear Regression r2-score:", r2_score(y_test_reg, y_pred_linreg))

Linear Regression MAE: 30.882256
Linear Regression RMSE: 65.255135
Linear Regression r2-score: 0.2435151523789395


## Hyperparameter Tuning

In [22]:
# Define hyperparameter space for Linear Regression
param_dist_lr = {
    "fit_intercept": [True, False]
}

In [23]:
# Initialize model
lr = LinearRegression()

# Use RandomizedSearchCV instead of GridSearchCV for efficiency
random_search_lr = RandomizedSearchCV(
    lr, param_distributions=param_dist_lr, cv=3, scoring="r2",
    n_iter=5, n_jobs=1, random_state=42
)

In [24]:
# Train model
random_search_lr.fit(X_train_reg, y_train_reg)

/opt/conda/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=3, estimator=LinearRegression(), n_iter=5, n_jobs=1,
                   param_distributions={'fit_intercept': [True, False]},
                   random_state=42, scoring='r2')

In [25]:
### ER - BEGIN ###
with open("../models/lin_reg_random_search.pkl", "wb") as file:
    pickle.dump(random_search_lr, file)

In [26]:
with open("../models/lin_reg_random_search.pkl", "rb") as file:
    random_search_lr = pickle.load(file)
### ER - END ###

In [27]:
# Best parameters and score
print("Best Linear Regression Parameters:", random_search_lr.best_params_)
print("Best Linear Regression Score:", random_search_lr.best_score_)

Best Linear Regression Parameters: {'fit_intercept': True}
Best Linear Regression Score: 0.2451144969267819


In [28]:
# Evaluate on test data
y_pred_lr = random_search_lr.best_estimator_.predict(X_test_reg)
print("Linear Regression R2 Score on Test Set:", r2_score(y_test_reg, y_pred_lr))

Linear Regression R2 Score on Test Set: 0.2435151523789395
